In [29]:
%load_ext autoreload
%autoreload 2
from hypopredict.fusion.ml_preproc import MLPreprocessor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

mlp = MLPreprocessor()
path = '/Users/alexxela/code/hypopredict/data/processed/ml_prepped_cv_splits_20251216_125112.pkl'
loaded_dict = MLPreprocessor().load_prepped_splits(path)
splits_prepped = loaded_dict['splits_prepped']

# carefully drop na values from each split
for SPLIT_INDEX in range(len(splits_prepped)):
    splits_prepped[SPLIT_INDEX][0]['target'] = splits_prepped[SPLIT_INDEX][1].copy()
    splits_prepped[SPLIT_INDEX][0].dropna(inplace=True)
    splits_prepped[SPLIT_INDEX] = (splits_prepped[SPLIT_INDEX][0].drop('target', axis=1),
                                   splits_prepped[SPLIT_INDEX][0]['target'].to_numpy())

# instantiate CV
from hypopredict.cv import CrossValidator
cv = CrossValidator(splits=splits_prepped)





# initialize XGBoost model
from xgboost import XGBClassifier
xgb = XGBClassifier(
    n_estimators=150,
    max_depth=1,
    reg_lambda=0.5,
    reg_alpha=0.5,
    learning_rate=0.150,
    eval_metric='logloss',
    random_state=17
)



from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", C=150.0, gamma="scale", probability=True))
])



# fit a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=150)



from sklearn.linear_model import LogisticRegression
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler
# fusion = Pipeline([
#     ("scaler", StandardScaler()),
#     ("logreg", LogisticRegression(max_iter=15000))
# ])

fusion = LogisticRegression(max_iter=15000)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
cv_results = cv.validate_model_cv(models=[xgb, svm, knn, fusion],
                                    fusion=True,
                                    splits_prepped=splits_prepped,
                                    resample=True,
                                    desired_pos_ratio=0.4,
                                    reduction_factor=0.7)

In [31]:
cv_results

{'val_pr_aucs': [0.962,
  0.335,
  0.991,
  0.51,
  0.995,
  0.225,
  0.99,
  0.139,
  0.988,
  0.06],
 'val_ave_precisions': [0.962,
  0.338,
  0.991,
  0.511,
  0.995,
  0.227,
  0.99,
  0.14,
  0.988,
  0.062]}